In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_14704\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
base_url = "https://www.justwatch.com"

In [5]:
# Python3 Program to Create list 
# with integers within given range 
  
def createYears(r1, r2):
    return [item for item in range(r1, r2+1)]
      
genres = ['act','cmy',  'doc', 'fnt', 'hrr', 'msc', 'rms', 'spt', 'wsn', 'ani', 'crm', 'drm', 'hst', 'fml', 'trl', 'scf', 'war', 'rly']
years = createYears(2020, 2023)


search_links = []
for i in genres:
    for j in years:
        link = f"https://www.justwatch.com/us/tv-shows?genres={i}&release_year_from={j}&release_year_until={j}"
        search_links.append(link)
        
len(search_links)

72

In [6]:
titleLinks = []
for link in search_links[:5]:
    print(link)
    chrome_options = Options()  
    chromedriver = webdriver.Chrome(ChromeDriverManager().install())
    with chromedriver as browser:
        browser.get(link)
        time.sleep(5)

        last_height = browser.execute_script("return document.body.scrollHeight")
        SCROLL_PAUSE_TIME = 5
        while True:
                # Scroll down to bottom
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = browser.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                
        page_source =browser.page_source
        soup=BeautifulSoup(page_source,'html.parser')
        productlist = soup.find("div",{"listlayout":"Grid"})
        titles = productlist.find_all("div", {"class":"title-list-grid__item"})
        for title in titles:
                    title_link = title.find("a").get('href')
                    title_link = base_url+title_link
                    if title_link not in titleLinks:
                        titleLinks.append(title_link)

https://www.justwatch.com/us/tv-shows?genres=act&release_year_from=2020&release_year_until=2020


C:\Users\user\AppData\Local\Temp\ipykernel_14704\2698527330.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chromedriver = webdriver.Chrome(ChromeDriverManager().install())


https://www.justwatch.com/us/tv-shows?genres=act&release_year_from=2021&release_year_until=2021
https://www.justwatch.com/us/tv-shows?genres=act&release_year_from=2022&release_year_until=2022
https://www.justwatch.com/us/tv-shows?genres=act&release_year_from=2023&release_year_until=2023
https://www.justwatch.com/us/tv-shows?genres=cmy&release_year_from=2020&release_year_until=2020


In [7]:
print(len(titleLinks))

1057


In [8]:
def get_detail(soup_object):
    labels  = []
    values = []
    table_info = soup_object.find("div", {"class":"title-info"})
    rows = table_info.find_all("div", {"class":"detail-infos"})
    for row in rows:
        label = row.find("div", {"class":"detail-infos__subheading"}).text.replace('\n',"")
        value = row.find("div", {"class":"detail-infos__value"}).text.replace('\n',"")
        labels.append(label)
        values.append(value)
        
    detail = {labels[i]: values[i] for i in range(len(labels))}
        
    return detail

def get_cast(soup_object):
    cast = []
    actors = soup_object.find_all("div", {"class":"title-credits__actor"})
    for actor in actors:
        actor_name = actor.text.replace('\n',"")
        cast.append(actor_name)
    return cast

class Pricing:
    def __init__(self, soup_object):
        self.soup_object = soup_object

    def get_stream_options(self):
        stream_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)


        return stream_list


    def get_rent_options(self):
        rent_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list

    def get_buy_options(self):
        buy_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list


In [9]:
def get_stream_options(soup_object):
        stream_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)
        return stream_list


def get_rent_options(soup_object):
        rent_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list
    
def get_buy_options(soup_object):
        buy_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list

In [44]:
def get_best(driver):
            driver.implicitly_wait(0.5)
            '''
            best = WebDriverWait(driver=driver, timeout=5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[3]/div/div[2]/div/div[1]/div[1]/div/div[1]'))
            )
            best.click()
            
            time.sleep(5)
            '''
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_best = get_stream_options(hun)
            buy_best = get_buy_options(hun)
            rent_best = get_rent_options(hun)
            
            options = {"stream_best": stream_best,
                    "rent_best":rent_best,
                    "buy_best":buy_best}
            
            return options
        
def get_free(driver, bts):
            time.sleep(5)
            bts[1].click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_free = get_stream_options(hun)
            buy_free = get_buy_options(hun)
            rent_free = get_rent_options(hun)
            
            
            options = {"stream_free": stream_free,
                    "rent_free":rent_free,
                    "buy_free":buy_free}
        
            return options
        
def get_sd(driver, bts):
            driver.implicitly_wait(0.5)
            bts[2].click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_sd = get_stream_options(hun)
            buy_sd = get_buy_options(hun)
            rent_sd = get_rent_options(hun)
            
            options = {"stream_sd": stream_sd,
                    "rent_sd":rent_sd,
                    "buy_sd":buy_sd}
        
            return options
        
def get_hd(driver, bts):
        driver.implicitly_wait(0.5)
            
        bts[3].click()
            
            
        time.sleep(5)
            
        page_source =driver.page_source
        
        hun=BeautifulSoup(page_source,'html.parser')
        stream_hd = get_stream_options(hun)
        buy_hd = get_buy_options(hun)
        rent_hd = get_rent_options(hun)
            
        options = {"stream_hd": stream_hd,
                    "rent_hd":rent_hd,
                    "buy_hd":buy_hd}
        
        return options
        
      
def get_4k(driver, bts):   
        driver.implicitly_wait(0.5)
        bts[4].click()
            
        time.sleep(5)
            
        page_source =  driver.page_source
        hun=BeautifulSoup(page_source,'html.parser')
        stream_4k = get_stream_options(hun)
        buy_4k = get_buy_options(hun)
        rent_4k = get_rent_options(hun)       
            
        options = {"stream_4k": stream_4k,
                    "rent_4k":rent_4k,
                    "buy_4k":buy_4k}
        
        return options

def get_season_links(soup_object):
    seasonLinks =[]
    seasons = soup_object.find("div", {"class":"horizontal-title-list"})
    seasons = seasons.find_all("div",{"class":"horizontal-title-list__item"} )
    for season in seasons:
        link = season.find("a").get("href")
        link_names  = season.find_all("div", {"class":"horizontal-title-list__item__caption"})
        for name in link_names:
            name = name.text.replace('\n',"")
            season = base_url+link
            season_urls = {
                name:season
            }
            seasonLinks.append(season_urls)
            
    return seasonLinks

def get_synopsis(driver):     
    synopsis = driver.find_element(By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/p').text
    return synopsis

In [45]:
def get_data(titleLinks):
    data = []
    for link in titleLinks[:30]:
            driver = webdriver.Chrome(ChromeDriverManager().install())
            with driver as driver:
                driver.get(link)
                
                banner = WebDriverWait(driver=driver, timeout=5).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[5]/div[2]/div[2]/button[1]'))
                    )
                
                banner.click()
                
                time.sleep(1)
                bts = driver.find_elements(By.CLASS_NAME,'jw-chip-button')
                
                try:
                    imdb_link = WebDriverWait(driver=driver, timeout=5).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[1]/div/aside/div[1]/div[3]/div[1]/div[2]/div/div[2]/a'))
                    )
               
                    imdb_link = imdb_link.get_attribute('href')
                except:
                    imdb_link = None
                
                page_source =  driver.page_source
                
                hun=BeautifulSoup(page_source,'html.parser')
                
                
                try:
                    detail = get_detail(hun)
                except:
                    detail = None
                    
                try:
                    title_name = hun.find("div", {"class":"title-block"})
                    title_name = title_name.find("h1").text.replace('\n',"")
                except:
                    title_name = None
                    
                try:
                    season_1_year = hun.find("span", {"class":"text-muted"}).text.replace('\n',"")
                except:
                    year = None
                
                try:
                    prices_best = get_best(driver)
                except:
                    prices_best = None
                
                try:
                    prices_free = get_free(driver,bts)
                except:
                    prices_free = None
                    
                try:
                    prices_sd = get_sd(driver, bts)
                except:
                    prices_sd = None
                    
                try:
                    prices_hd = get_hd(driver, bts)
                except:
                    prices_hd = None
                    
                try:   
                    prices_4k = get_4k(driver, bts)
                except:
                    prices_4k = None

                try:
                    season_urls = get_season_links(hun)
                except:
                    season_urls = None
        
                title_url = link
                
                tv = {"title":title_name, "season_1_year":season_1_year, "season_urls":season_urls,"detail":detail, "4k prices":prices_4k, "hd prices":prices_hd, "sd prices":prices_sd, "free":prices_free, "best prices":prices_best,"imdb_link":imdb_link,"tite url": title_url}
        
                data.append(tv)
        
    return  data
        
        
        

In [46]:
Data = get_data(titleLinks)

C:\Users\user\AppData\Local\Temp\ipykernel_14704\3468145898.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [33]:
import pandas as pd
df = pd.DataFrame.from_dict(Data)
df

,title,season_1_year,season_urls,detail,4k prices,hd prices,sd prices,free,best prices,imdb_link,tite url
0,Star Trek: Lower Decks,(2020),[{' Season 3 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 81% 7.3 (16k) ', 'Genres': 'Scie...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}",{'stream_hd': [{'Paramount+ Amazon Channel': '...,{'stream_sd': [{'Paramount+ Amazon Channel': '...,{'stream_free': [{'The Roku Channel': '2 Seaso...,{'stream_best': [{'Paramount+ Amazon Channel':...,https://www.imdb.com/title/tt9184820/?ref_=ref...,https://www.justwatch.com/us/tv-show/star-trek...
1,Snowpiercer,(2020),[{' Season 3 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 80% 6.9 (54k) ', 'Genres': 'Scie...",None,None,None,None,None,https://www.imdb.com/title/tt6156584/?ref_=ref...,https://www.justwatch.com/us/tv-show/snowpiercer
2,Star Trek: Picard,(2020),[{' Season 2 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 81% 7.3 (70k) ', 'Genres': 'Scie...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}",{'stream_hd': [{'Paramount Plus': '2 Seasons H...,{'stream_sd': [{'Paramount Plus': '2 Seasons '...,{'stream_free': [{'Paramount Plus': '1 Season ...,{'stream_best': [{'Paramount Plus': '2 Seasons...,https://www.imdb.com/title/tt8806524/?ref_=ref...,https://www.justwatch.com/us/tv-show/star-trek...
3,Close Enough,(2020),[{' Season 3 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 88% 7.8 (11k) ', 'Genres': 'Come...",None,None,None,None,None,https://www.imdb.com/title/tt6994156/?ref_=ref...,https://www.justwatch.com/us/tv-show/close-enough
4,Jujutsu Kaisen,(2020),[{' Season 2 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 95% 8.6 (55k) ', 'Genres': 'Acti...",None,None,None,None,None,https://www.imdb.com/title/tt12343534/?ref_=re...,https://www.justwatch.com/us/tv-show/jujutsu-k...
5,DC's Stargirl,(2020),[{' Season 3 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 84% 7.2 (17k) ', 'Genres': 'Scie...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [{'HBO Max': '2 Seasons HD '}, {...","{'stream_sd': [{'HBO Max': '2 Seasons '}, {'Th...","{'stream_free': [{'The CW': '2 Seasons '}, {'C...","{'stream_best': [{'HBO Max': '2 Seasons HD '},...",https://www.imdb.com/title/tt8722888/?ref_=ref...,https://www.justwatch.com/us/tv-show/stargirl
6,Gangs of London,(2020),[{' Season 2 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 87% 8.1 (28k) ', 'Genres': 'Crim...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}",{'stream_hd': [{'Amazon Prime Video': '1 Seaso...,"{'stream_sd': [{'fuboTV': '1 Season '}, {'AMC+...",{'stream_free': [{'Amazon Prime Video': '1 Sea...,{'stream_best': [{'Amazon Prime Video': '1 Sea...,https://www.imdb.com/title/tt7661390/?ref_=ref...,https://www.justwatch.com/us/tv-show/gangs-of-...
7,Tehran,(2020),[{' Season 2 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 83% 7.5 (17k) ', 'Genres': 'Dram...",None,None,None,None,None,https://www.imdb.com/title/tt10577736/?ref_=re...,https://www.justwatch.com/us/tv-show/tehran
8,Jurassic World: Camp Cretaceous,(2020),[{' Season 5 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 86% 7.5 (8k) ', 'Genres': 'Actio...",None,None,None,None,None,https://www.imdb.com/title/tt10436228/?ref_=re...,https://www.justwatch.com/us/tv-show/jurassic-...
9,The Owl House,(2020),[{' Season 3 ': 'https://www.justwatch.com/us/...,"{'Rating': ' 94% 8.2 (10k) ', 'Genres': 'Fant...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}",{'stream_hd': [{'Disney Plus': '2 Seasons HD '...,"{'stream_sd': [{'Disney Plus': '2 Seasons '}, ...","{'stream_free': [], 'rent_free': [], 'buy_free...",{'stream_best': [{'Disney Plus': '2 Seasons HD...,https://www.imdb.com/title/tt8050756/?ref_=ref...,https://www.justwatch.com/us/tv-show/the-owl-h...


In [20]:
k_df = df['4k prices'].apply(pd.Series)
h_df = df['hd prices'].apply(pd.Series)
s_df = df['sd prices'].apply(pd.Series)
f_df = df['free'].apply(pd.Series)
b_df = df['best prices'].apply(pd.Series)
d_df = df['detail'].apply(pd.Series)
d_df

,Rating,Genres,Runtime,Age rating
0,81% 7.3 (16k),"Science-Fiction , Comedy , Animation , Action ...",25min,TV-14


In [21]:
df = pd.concat([df, k_df.reindex(df.index)], axis=1)
df = pd.concat([df, h_df.reindex(df.index)], axis=1)
df = pd.concat([df, s_df.reindex(df.index)], axis=1)
df = pd.concat([df, f_df.reindex(df.index)], axis=1)
df = pd.concat([df, b_df.reindex(df.index)], axis=1)
df = pd.concat([df, d_df.reindex(df.index)], axis=1)

df_c = df.drop(['4k prices', 'hd prices', 'sd prices', 'free', 'best prices', 'detail'], axis=1)

In [ ]:
df.to_csv("2022_action.csv")

In [43]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.justwatch.com/us/tv-show/snowpiercer")
                
banner = WebDriverWait(driver=driver, timeout=5).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[5]/div[2]/div[2]/button[1]'))
                    )
                
banner.click()
                
time.sleep(1)
bts = driver.find_elements(By.CLASS_NAME,'jw-chip-button')
bts[1].click()

C:\Users\user\AppData\Local\Temp\ipykernel_14704\3261235698.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
get_season_links(hun)